# Deep Recurrent Learning (N-BEATS) vs. Traditional Machine Learning
## Advanced Machine Learning in Finance - Individual Project

The aim of this project is to compare the performance of two machine learning models, N-Beats and RandomForests, for predicting sales figures. The challenge is to use available data that has a correlation with the sales of various companies, such as credit card transactions to predict the quarterly sales figures.

The goal is to compare the accuracy of predictions for the current quarter, as well as the next couple of quarters, between N-Beats and RandomForests.

N-Beats is a deep learning model designed specifically for time-series forecasting, while RandomForests is a popular tree-based model used for various types of predictive modeling. This project aim to give insights into comparative forecasting results from these two models and their applications in the field of sales prediction.

The outcome of this project will be a comparison between the two models, including a discussion of the results and their implications for businesses and organizations looking to improve their financial forecasting capabilities.

In [ ]:
import darts
import numpy as np
import data_wrangling as dw
import time_series_preprocessing as tsp
import matplotlib.pyplot as plt
import visualisations as vis
from darts.models import NBEATSModel
from darts.models import BlockRNNModel, RandomForest, XGBModel
from darts.utils.likelihood_models import QuantileRegression
from darts.metrics import mape, rmse
from skopt import gp_minimize
from skopt.space import Integer
from skopt.utils import use_named_args

# Data Extraction

In [ ]:
revenue = dw.create_df('revenue.xlsx')
spendings = dw.create_df('spend_amounts_aggregated.xlsx')
df = dw.merge_dataframes(df_left=spendings, df_right=revenue)
df = dw.encode_index(df, 'mic')
df = dw.add_time_cols(df)
df = dw.remove_short_series(df)
df = dw.remove_missing_ground_truth(df)
df.head()

## Data Exploration

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
# total number of companies
len(df['ticker'].unique())

In [ ]:
df.isna().sum()

In [ ]:
dw.print_nans_companies(df)

We see that the vast majority of missing vals are happening in the column 'nw_total_sales_b_total' for the indices in some form of subset of [0,1,2,3] so that it makes sense to impute these values to avoid missing large amounts of data.

In [ ]:
vis.plot_scatter_log(df,'nw_total_sales_a_total', 'nw_total_sales_b_total')

In [ ]:
vis.plot_hist(df, 'nw_total_sales_a_total','nw_total_sales_b_total' )


We see that the distributions are quite similar but shifted, and that they have a fairly linear relationship when we apply the log. Thus, a linear least squares approach is justified. Including the other columns give a higher degree of freedom, but it could be justified to just use these columns up against each other.

In [ ]:
df = dw.impute_nans_singular_column(df, col='nw_total_sales_b_total' ,plot=True, max_plots=3)


Based on the plots a LLS solution looks sensible to the following time series, and thus handles many of the missing datapoints. The accuracy of these are however unknown. Bayesian regression can be performed to quantify uncertainty on the imputation on the missing values. To avoid noisy inputs, one could also perform lasso/ridge regression to shrink mutual information features and reduce high variance issues.   

In [ ]:
df.isna().sum()

We see that the Actual and Estimated sales for fiscal quarters have a decent amount of NaN values. Thus, we can statistically inspect them as well, and see if there are favorable methods of imputation of these.

In [ ]:
vis.plot_hist(df, 'Sales_Actual_fiscal', 'Sales_Estimate_fiscal')

The distributions have a very large overlap. Note that the distribution is a concetanation of all companies, so even though this looks normal, it is not necessarily the case for the distribution of actual sales and estimated sales with respect to the company in question.

In [ ]:
vis.plot_scatter_log(df, 'Sales_Actual_fiscal', 'Sales_Estimate_fiscal')

From this it looks as they are very linearly correlated. Let's inspect how it looks like for respective companies

In [ ]:
vis.plot_sales_comparison(df, max_plots=3)

It is evident that there are many instances for which the fiscal sales are missing for both estimate and actual, for that reason, we should apply an algorithm that can impute the missing values well. When both have NaNs at the same index over a time span ~5 there are uncertainty with doing rolling averages as there are fluctuations between quarters for many companies. The suggested idea would be to that of a KNN or KD-Ball algorithm to replace these instances. However, for the instances when there are rows when one or the other is present, it makes sense to impute values based on the other. Applying a normal centered at the no NaN column value at the corresponding index with a standard deviation that is the square root of the absolute difference of numeric rows seems like a intuitive and good solution based on their distribution. Also, when there is just a singular column of these with NaN values after doing the computations, we can perform LLS like earlier on those columns, given linearity exists in the dataset for these instances.

In [ ]:
df = dw.fiscal_sales_imputation(df, plot=False)

### Remaining NaNs

In [ ]:
df.isna().sum()

For many of the total sales in A and B these are overlapping. A strategy here could be to use KNN / KD-Balls, Decision Trees or other ML algorithms for imputations

In [ ]:
dw.print_nans_companies(df)

In [ ]:
df = dw.impute_nans_singular_column(df, col='nw_total_sales_b_total',plot=True, max_plots=3)

It can seem as this is cause for some anomolies when performing LLS. Let's create a pipeline to detect anomolies.

In [ ]:
df.ticker.unique()

In [ ]:
df = dw.impute_nans_singular_column(df)

In [ ]:
df = df.dropna(how='any')
df = dw.remove_short_series(df)
print(df.shape, '\n')
print(df.isna().sum())


# Creating new columns with high correlation to target

In [ ]:
df = dw.add_proportion_ab(df)
df = dw.add_quarterly_yoy(df)
df = dw.add_prod(df)

In [ ]:
df.ticker.unique()

In [ ]:
vis.plot_correlation_matrix(df)

The **key** to understanding here is that the high uncorrelation comes from the fact that we have a dataframe containing companies in different market segments with completely different scales to their sales by season, size of company etc., so looking at the whole dataframe gives a misleading correlation to the actual ground truth. If we take a single company, we can see that the correlation between columns will change drastically.

In [ ]:
brand = 'AMZN'
df_brand = df[df['ticker'] == brand]
vis.plot_correlation_matrix(df_brand)

Therefore, the removal of columns is done by the respective ticker. If the overall mean covariance across all companies is below a threshold (set to 0.1), we remove it. Luckily for us there seem to be a very high correlation between actual sales to all other features

In [ ]:
corr = df.groupby('ticker').apply(lambda x: x.corrwith(x['Sales_Actual_fiscal'], numeric_only=True)).mean()
corr = corr.sort_values(ascending=False)
corr


In [ ]:
# function to drop low correlation features
df = dw.drop_low_correlation_features(df)

In [ ]:
df_brand = df[df.ticker == 'AMZN']
vis.plot_correlation_matrix(df_brand)

The impact of covid and other macro economic events can have a significant impact on the sales. This we see in the reflection of the year's correlation with sales. An underlying factor for why this is so high is probably due to the effect of pre-mid-post COVID.

# Building Series in Darts
Note that the data frame must be thoroughly processed before starting this process, and keep in mind to that the model actually does handle NaNs implicitly. So, the thorough removal may not be neccessary. Especially, the line when all NaNs are dropped. Darts also has it's own models for imputations, but due to the structure of the data in this project, manual imputation using simple statistics was done.

As we create time series in darts, our model can support stationary covariates. As there is significant differences between companies in their sales volume, seasonalities, etc. we include this information to the model we are about to create. Thus, we must encode the stationary covariates, which we can do by mapping the tickers to a numeric value. To avoid numeric instability to the mapping, we do not do integer encoding as we have >200 companies to evaluate. This is to avoid any arbitrary ordering of assigning integer values. To have the model capture the stationary covariates there are two alternatives that seem reasonable:  
  
1. One Hot Encoding:   
Represent the tickers as one hot encoded covariates added to the function.
  
2. Mapping:  
Map the company to a range of somewhat similar float values to avoid numerical instability 
 
Now, because the number of companies exceeds the number of rows by a significant amount, it is uncertain whether the network will carry an advantage of adding so many stationary covariates. Thus, I have gone with option 2.

In [ ]:
df, inv_mapper = dw.encode_float(df, 'ticker')

Functions above help extract appropriate covariates and target for each company within the data frame

### Converting the data frame into series for each company

In [ ]:
series_dict = tsp.dict_series(df, inv_mapper)
past_cov, future_cov, target, tickers = tsp.past_future_split(series_dict, n_preds=2)
past_cov, future_cov, target, tickers = tsp.match_input_length(past_cov, future_cov, target, tickers, min_train = 12)
past_cov, future_cov, target, scaler_cov, scaler_target = tsp.scale_series(past_cov, future_cov, target)

In the below line all lists have the corresponding index to the respective company.
I.e., If we look at;  

 ```train_past_cov[i], train_target[i], train_future_cov[i], tickers_train[i], scaler_cov_train[i], scaler_target_train[i]```

It corresponds to the same company $\forall i$

In [ ]:
# Split the data into training and testing sets
train_past_cov, test_past_cov, \
train_future_cov, test_future_cov, \
train_target, test_target, \
tickers_train, tickers_test, \
scaler_cov_train, scaler_cov_test, \
scaler_target_train, scaler_target_test \
= tsp.train_test_split(
    past_cov,  # input feature data for past time steps in the training set
    future_cov,  # input feature data for future time steps in the training set
    target,  # target variable data in the training set
    tickers,  # ticker symbols for the data in the training set
    scaler_cov,  # scaler object to normalize the input feature data in the training set
    scaler_target,  # scaler object to normalize the target variable data in the training set
    test_size=0.2  # the proportion of the data to be used for testing
)

In [ ]:
# remove points to predict from train and test data
train_target_input = tsp.remove_n_predictions(train_target, train_future_cov)
test_target_input = tsp.remove_n_predictions(test_target, test_future_cov)

## N-Beats settings

In [ ]:
epochs = 300                        # number of epochs
n_stacks = 3                        # number of stacks in model
n_blocks = 9                        # number of blocks in model        
layer_width = 128                   # numer of weights in FC layer
lr = 1e-3                           # learning rate
val_wait = 1                        # epochs to wait before evaluating the loss on the test/validation set
seed = 42                           # random seed for regenerating results
n_jobs = -1                         # parallel processors to use;  -1 = all processors
input_length = len(past_cov[0])     # length of the input sequence
output_length = len(future_cov[0])  # length of the output sequence

# quantiles for QuantileRegression argument
quantiles = [0.01, 0.1, 0.2, 0.5, 0.8, 0.9, 0.99]

# lower and upper quantiles for predictions
QL1, QL2 = 0.01, 0.05 
QU1, QU2 = 1 - QL1, 1 - QL2 
# labels for plotting
labelQ1 = f'{int(QU1 * 100)} / {int(QL1 * 100)} percentile band'
labelQ2 = f'{int(QU2 * 100)} / {int(QL2 * 100)} percentile band'


In [ ]:
model_nbeats = NBEATSModel(input_chunk_length=len(train_past_cov[0]), 
                     output_chunk_length=len(train_future_cov[0]),
                     num_stacks=n_stacks,
                     num_blocks=n_blocks,
                     layer_widths=layer_width,
                     n_epochs=epochs,
                     likelihood = QuantileRegression(quantiles),
                     optimizer_kwargs = {"lr" : 1e-3},
                     generic_architecture=True,
                     trend_polynomial_degree=2,
                     random_state=seed,
                     nr_epochs_val_period=val_wait,
                     #pl_trainer_kwargs = {"accelerator": "cpu", "devices": 8 } 
                     )
predictor_nbeats = model_nbeats.fit(series=train_target, 
                       past_covariates=train_past_cov,
                       verbose=False,
                       epochs=epochs)

In [ ]:
# TODO make a function for predicting using darts predictor - see docs
pred_train = predictor_nbeats.predict(n=2,
                                series=train_target_input,
                                past_covariates=train_past_cov,
                                n_jobs=-1,
                                mc_dropout=True,
                                )

                       

In [ ]:
pred_test = predictor_nbeats.predict(n=2,
                                series=test_target_input,
                                past_covariates=test_past_cov,
                                n_jobs=-1,
                                mc_dropout=True,
                                )

Right now the uncertainty bands do not represent anything. By retraining the model n times "large enough" for assuming normality in the predictions we can assert an uncertainty band to the predictions. This is omitted due to the training time involved in doing so.

# Traininig plots

In [ ]:
vis.plot_predictions(pred_train, train_target, scaler_target_train, tickers_train)

# Test plots

In [ ]:
vis.plot_predictions(pred_test, test_target, scaler_target_test, tickers_test)

# Hyperparameter Tuning


### N-Beats / RNN (Architecture Design)  
https://unit8co.github.io/darts/examples/17-hyperparameter-optimization.html
* n_layers
* n_stacks
* input_chunk_length (lookback window of the model)
* output_chunk_length (how many steps into the future we make predictions)
* n past covariates to use
* n epochs in training
* layer width


In [ ]:
import torch

In [ ]:
torch.mean(torch.tensor([rmse(target[-2:], pred) for target, pred in zip(train_target, pred_train)]))


In [ ]:
import numpy as np
from scipy.optimize import minimize

def objective_nbeats(params, train_target, train_past_cov, seed=42):
    """ 
    Optimization function for hyperparameter tuning

    Args:
        params (tuple): hyperparameters to optimize
        train_target (TimeSeries): training target
        train_past_cov (TimeSeries): training past covariates
        val_target_input (TimeSeries): validation target
        val_past_cov (TimeSeries): validation past covariates
        seed (int): random seed for reproducibility

    Returns:
        float: validation loss
    """
    n_stacks, n_blocks, layer_width, epochs, val_wait = map(int, params)
    quantiles = [0.1, 0.5, 0.9]
    model = NBEATSModel(input_chunk_length=input_length, 
                        output_chunk_length=output_length,
                        num_stacks=n_stacks,
                        num_blocks=n_blocks,
                        layer_widths=layer_width,
                        n_epochs=epochs,
                        likelihood=QuantileRegression(quantiles),
                        optimizer_kwargs={"lr": 1e-3},
                        generic_architecture=True,
                        trend_polynomial_degree=2,
                        random_state=seed,
                        nr_epochs_val_period=val_wait,
                        )
    model.fit(series=train_target,
              past_covariates=train_past_cov,
              epochs=epochs,
              verbose=True)
    preds_val = model.predict(n=2,
                              series=test_target_input,
                              past_covariates=test_past_cov)
    
    return np.mean([rmse(target[-2:], pred) for target, pred in zip(test_target, preds_val)])
    

In [ ]:
from skopt import gp_minimize
from skopt.space import Integer
from skopt.utils import use_named_args

space = [
         Integer(2, 5, name='num_stacks'),
         Integer(4, 10, name='num_blocks'),
         Integer(64, 256, name='layer_width'),
         Integer(50, 100, name='n_epochs'),
         Integer(1, 3, name='nr_epochs_val_period')]

# Define the objective function to optimize
@use_named_args(space)
def objective(num_stacks, num_blocks, layer_width, n_epochs, nr_epochs_val_period):
    params = (num_stacks, num_blocks, layer_width, n_epochs, nr_epochs_val_period)
    return objective_nbeats(params, train_target, train_past_cov)

# Use gp_minimize to optimize the objective function
result = gp_minimize(objective, space, n_calls=50)

# Print the best hyperparameters found
print("Best parameters: ", result.x)

In [ ]:
input_length = len(train_past_cov[0])
output_length = len(train_future_cov[0])
target_length = len(train_target[0])
input_length, output_length, target_length

# XGBoost

In [ ]:
from darts.models import XGBModel

xgb_model = XGBModel(
    lags=input_length,
    lags_past_covariates=input_length,
    lags_future_covariates=[output_length],
    output_chunk_length=output_length,
    likelihood='quantile',
    n_jobs=-1,
    quantiles=[0.05, 0.5, 0.95],
    random_state=seed,
    multi_models=False,
)

xgb_predictor = xgb_model.fit(series=train_target,
                              past_covariates=train_past_cov,
                              future_covariates=train_future_cov,
                              verbose=True)
